Based on the papers:

@article{wood2021trading,
  title={Trading with the Momentum Transformer: An Intelligent and Interpretable Architecture},
  author={Wood, Kieran and Giegerich, Sven and Roberts, Stephen and Zohren, Stefan},
  journal={arXiv preprint arXiv:2112.08534},
  year={2021}
}

@article {Wood111,
	author = {Wood, Kieran and Roberts, Stephen and Zohren, Stefan},
	title = {Slow Momentum with Fast Reversion: A Trading Strategy Using Deep Learning and Changepoint Detection},
	volume = {4},
	number = {1},
	pages = {111--129},
	year = {2022},
	doi = {10.3905/jfds.2021.1.081},
	publisher = {Institutional Investor Journals Umbrella},
	issn = {2640-3943},
	URL = {https://jfds.pm-research.com/content/4/1/111},
	eprint = {https://jfds.pm-research.com/content/4/1/111.full.pdf},
	journal = {The Journal of Financial Data Science}
}

In [2]:
!pip install gpflow


  Using cached gpflow-2.9.2-py3-none-any.whl.metadata (13 kB)
  Using cached check_shapes-1.1.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached tensorflow_probability-0.25.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached dropstackframe-0.1.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached lark-1.2.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached gpflow-2.9.2-py3-none-any.whl (392 kB)
Using cached check_shapes-1.1.1-py3-none-any.whl (45 kB)
Using cached tensorflow_probability-0.25.0-py2.py3-none-any.whl (7.0 MB)
   ---------------------------------------- 0.0/390.0 MB ? eta -:--:--
    --------------------------------------- 8.4/390.0 MB 47.2 MB/s eta 0:00:09
   -- ------------------------------------- 22.0/390.0 MB 55.8 MB/s eta 0:00:07
   --- -------------------

In [ ]:
import pandas as pd
import os
import numpy as np
import gpflow
import tensorflow as tf
import datetime as dt
from gpflow.kernels import ChangePoints, Matern32
from typing import Dict, List, Optional, Tuple, Union
from tensorflow_probability import bijectors as tfb
from sklearn.preprocessing import StandardScaler

Kernel = gpflow.kernels.base.Kernel
MAX_ITERATIONS = 50

: 

In [3]:
class ChangePointDetection(ChangePoints):
    def __init__(
            self,
            kernels: Tuple[Kernel, Kernel],
            location: float,
            interval: Tuple[float, float],
            steepness: float = 1.0,
            name: Optional[str] = None
    ):
        if location < interval[0] or location > interval[1]:
            raise ValueError(
                "Location {loc} is not in range [{low},{high}]".format(
                    loc=location, low=interval[0], high=interval[1]
                )
            )
        locations = [location]
        super().__init__(
            kernels = kernels, locations = locations, steepness = steepness, name=name
        )

        affine = tfb.Shift(tf.cast(interval[0], tf.float64))(
            tfb.Scale(tf.cast(interval[1] - interval[0], tf.float64))
        )

        self.locations = gpflow.Parameter(
            locations, transform=tfb.Chain([affine, tfb.Sigmoid()]), dtype = tf.float64
        )

        def _sigmoids(self, X: tf.Tensor):
            locations = tf.reshape(self.locations, (1, 1, -1))
            steepness = tf.reshape(self.steepness, (1, 1, -1))
            return tf.sigmoid(steepness * (X[:, :, None] - locations))

In [4]:
def fit_matern_kernel(
        time_series_data: pd.DataFrame,
        variance: float = 1.0,
        lengthscale: float = 1.0,
        likelihood_variance: float = 1.0
):
    
    model = gpflow.models.GPR(
        data = (
            time_series_data.loc[:, ["X"]].to_numpy(),
            time_series_data.loc[:, ["Y"]].to_numpy()
        ),
        kernel = Matern32(variance=variance, lengthscales=lengthscale),
        noise_variance=likelihood_variance
    )

    optimizer = gpflow.optimizers.Scipy()
    nlml = optimizer.minimize(
        model.training_loss, model.trainable_variables, options=dict(maxiter=MAX_ITERATIONS)
    ).fun
    parameters = {
        "kM_variance": model.kernel.variance.numpy(),
        "kM_lengthscales": model.kernel.lengthscales.numpy(),
        "kM_likelihood_variance": model.likelihood.variance.numpy()
    }

    return nlml, parameters

In [5]:
def fit_changepoint_kernel(
        time_series_data: pd.DataFrame,
        k1_variance: float = 1.0,
        k1_lengthscale: float = 1.0,
        k2_variance: float = 1.0,
        k2_lengthscale: float = 1.0,
        kC_likelihood_variance = 1.0,
        kC_changepoint_location = None,
        kC_steepness = 1.0
):
    if not kC_changepoint_location:
        kC_changepoint_location = (
            time_series_data["X"].iloc[0] + time_series_data["X"].iloc[-1]
        ) / 2.0

    model = gpflow.models.GPR(
        data=(
            time_series_data.loc[:, ["X"]].to_numpy(),
            time_series_data.loc[:, ["Y"]].to_numpy()
        ),
        kernel = ChangePointDetection(
            [
                Matern32(variance=k1_variance, lengthscales=k1_lengthscale),
                Matern32(variance=k2_variance, lengthscales=k2_lengthscale)
            ],
            location=kC_changepoint_location,
            interval=(time_series_data["X"].iloc[0], time_series_data["X"].iloc[-1]),
            steepness=kC_steepness
        )
    )
    model.likelihood.variance.assign(kC_likelihood_variance)
    
    optimizer = gpflow.optimizers.Scipy()
    nlml = optimizer.minimize(
        model.training_loss, model.trainable_variables, options=dict(maxiter=MAX_ITERATIONS)
    ).fun
    changepoint_location = model.kernel.locations[0].numpy()
    parameters = {
        "k1_variance": model.kernel.kernels[0].variance.numpy().flatten()[0],
        "k1_lengthscale": model.kernel.kernels[0].lengthscales.numpy().flatten()[0],
        "k2_variance": model.kernel.kernels[1].variance.numpy().flatten()[0],
        "k2_lengthscale": model.kernel.kernels[1].lengthscales.numpy().flatten()[0],
        "kC_likelihood_variance": model.likelihood.variance.numpy().flatten()[0],
        "kC_changepoint_location": changepoint_location,
        "kC_steepness": model.kernel.steepness.numpy()
    }

    return changepoint_location, nlml, parameters

In [6]:
def changepoint_severity(
     kC_nlml: Union[float, List[float]], 
     kM_nlml: Union[float, List[float]]
):
    normalized_nlml = kC_nlml - kM_nlml
    return 1 - 1 / (np.mean(np.exp(-normalized_nlml)) + 1)

In [7]:
def changepoint_loc_and_score(
    time_series_data_window: pd.DataFrame,
    kM_variance: float = 1.0,
    kM_lengthscale: float = 1.0,
    kM_likelihood_variance: float = 1.0,
    k1_variance: float = None,
    k1_lengthscale: float = None,
    k2_variance: float = None,
    k2_lengthscale: float = None,
    kC_likelihood_variance: float = None,
    kC_changepoint_location: float = None,
    kC_steepness=1.0
):
    time_series_data = time_series_data_window.copy()
    Y_data = time_series_data[["Y"]].values
    time_series_data[["Y"]] = StandardScaler().fit(Y_data).transform(Y_data)

    
    if kM_variance == kM_lengthscale == kM_likelihood_variance == 1.0 :
        (kM_nlml, kM_params) = fit_matern_kernel(time_series_data)
    else:
        (kM_nlml, kM_params) = fit_matern_kernel(time_series_data, kM_variance, kM_lengthscale, kM_likelihood_variance)
    
    is_cp_location_default = (
        (not kC_changepoint_location)
        or kC_changepoint_location < time_series_data["X"].iloc[0]
        or kC_changepoint_location > time_series_data["X"].iloc[-1]
    )

    if is_cp_location_default:
        kC_changepoint_location = (
            time_series_data["X"].iloc[-1] + time_series_data["X"].iloc[0]
        ) / 2.0

    if not k1_variance:
        k1_variance = kM_params["kM_variance"]

    if not k1_lengthscale:
        k1_lengthscale = kM_params["kM_lengthscales"]

    if not k2_variance:
        k2_variance = kM_params["kM_variance"]

    if not k2_lengthscale:
        k2_lengthscale = kM_params["kM_lengthscales"]

    if not kC_likelihood_variance:
        kC_likelihood_variance = kM_params["kM_likelihood_variance"]


    if (k1_variance == k1_lengthscale == k2_variance == k2_lengthscale == kC_likelihood_variance == kC_steepness == 1.0) and is_cp_location_default:
        (changepoint_location, kC_nlml, kC_params) = fit_changepoint_kernel(time_series_data)
    else:
        (changepoint_location, kC_nlml, kC_params) = fit_changepoint_kernel(
            time_series_data,
            k1_variance=k1_variance,
            k1_lengthscale=k1_lengthscale,
            k2_variance=k2_variance,
            k2_lengthscale=k2_lengthscale,
            kC_likelihood_variance=kC_likelihood_variance,
            kC_changepoint_location=kC_changepoint_location,
            kC_steepness=kC_steepness,
        )
    
    cp_score = changepoint_severity(kC_nlml, kM_nlml)
    cp_loc_normalised = (time_series_data["X"].iloc[-1] - changepoint_location) / (
        time_series_data["X"].iloc[-1] - time_series_data["X"].iloc[0]
    )

    return cp_score, changepoint_location, cp_loc_normalised, kM_params, kC_params
    


In [8]:
def run_CPD(
    time_series_data: pd.DataFrame,
    lookback_window_length: int,
    start_date: dt.datetime = None,
    end_date: dt.datetime = None,
    use_kM_hyp_to_initialize_kC=True
):
    if start_date and end_date:
        first_window = time_series_data.loc[:start_date].iloc[
            -(lookback_window_length + 1) :, :
        ]
        remaining_data = time_series_data.loc[start_date:end_date, :]
        if remaining_data.index[0] == start_date:
            remaining_data = remaining_data.iloc[1:, :]
        else:
            first_window = first_window.iloc[1:]
        time_series_data = pd.concat([first_window, remaining_data]).copy()
    else:
        raise Exception("Pass start and end date.")

    time_series_data["date"] = time_series_data.index
    time_series_data = time_series_data.reset_index(drop=True)

    results = []
    for window_end in range(lookback_window_length + 1, len(time_series_data)):
        ts_data_window = time_series_data.iloc[
            window_end - (lookback_window_length + 1) : window_end
        ][["date", "daily_returns"]].copy()
        ts_data_window["X"] = ts_data_window.index.astype(float)
        ts_data_window = ts_data_window.rename(columns={"daily_returns": "Y"})
        time_index = window_end - 1
        window_date = ts_data_window["date"].iloc[-1].strftime("%Y-%m-%d")

        if use_kM_hyp_to_initialize_kC:
            cp_score, cp_loc, cp_loc_normalised, _, _ = changepoint_loc_and_score(ts_data_window)
        else:
            cp_score, cp_loc, cp_loc_normalised, _, _ = changepoint_loc_and_score(
                    ts_data_window,
                    k1_lengthscale=1.0,
                    k1_variance=1.0,
                    k2_lengthscale=1.0,
                    k2_variance=1.0,
                    kC_likelihood_variance=1.0,
                )
        results.append([window_date, time_index, cp_loc, cp_loc_normalised, cp_score]) 

    results_df = pd.DataFrame(results, columns=["date", "t", "cp_location", "cp_location_norm", "cp_score"]) 
    return results_df

In [12]:
import wrds

#Change this to get the dates and companies that we are using


# Step 1: Connect to WRDS
conn = wrds.Connection()

# Step 2: Define the ticker and date range
ticker = 'NEM'
start_date = '2017-01-01'
end_date = '2023-12-31'

# Step 3: Query CRSP for the selected stock
query = f"""
    SELECT a.permno, a.date, a.ret, b.ticker 
    FROM crsp.dsf AS a
    JOIN crsp.dsenames AS b 
    ON a.permno = b.permno 
    WHERE b.ticker = '{ticker}' 
    AND b.shrcd BETWEEN 10 AND 12
    AND b.exchcd BETWEEN 1 AND 3
    AND a.date BETWEEN '{start_date}' AND '{end_date}'
    AND a.date >= b.namedt
    AND a.date <= b.nameendt
"""

# Fetch the data
msft_data = conn.raw_sql(query)

# Step 4: Process the data
msft_data['date'] = pd.to_datetime(msft_data['date'])
msft_data.set_index('date', inplace=True)
msft_data = msft_data[['ret']].rename(columns={'ret': 'daily_returns'})

# Step 5: Close the connection
conn.close()

# Step 6: Display the data
print(msft_data.head())
print("\nData range:", msft_data.index.min(), "to", msft_data.index.max())



WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
            daily_returns
date                     
2017-01-03       0.016437
2017-01-04       0.009529
2017-01-05       0.046053
2017-01-06      -0.031447
2017-01-09      -0.001694

Data range: 2017-01-03 00:00:00 to 2023-12-29 00:00:00


In [13]:
lookback_window_length = 21
start_date = dt.datetime(2017, 1, 1)
end_date = dt.datetime(2023, 12, 31)


result = run_CPD(
    time_series_data=msft_data,
    lookback_window_length=lookback_window_length,
    start_date=start_date,
    end_date=end_date,
    use_kM_hyp_to_initialize_kC=True
)

result.to_csv('C:/Users/Maxim/Desktop/DDMIF/Changepoint files/nem_lbw21.csv', index=False)

In [31]:
result

,date,t,cp_location,cp_location_norm,cp_score
0,2017-02-02,21,16.641588,0.207543,0.878405
1,2017-02-03,22,15.236636,0.322065,0.865126
2,2017-02-06,23,18.081238,0.234227,0.954077
3,2017-02-07,24,18.049769,0.283344,0.991005
4,2017-02-08,25,18.165632,0.325446,0.999898
...,...,...,...,...,...
1733,2023-12-21,1754,1748.195791,0.276391,0.926757
1734,2023-12-22,1755,1748.933988,0.288858,0.951923
1735,2023-12-26,1756,1748.513029,0.356522,0.920551
1736,2023-12-27,1757,1748.541038,0.402808,0.857886


In [32]:
result.to_csv('C:/Users/Maxim/Desktop/DDMIF/Changepoint files/alco_lbw21.csv', index=False)